In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import re

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT3/at_3/adv_mla_at3/data/combined_output_25%.csv')

In [3]:
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,22253e139fd451062f9707d4b772eccb,2022-05-01,2022-05-18,OAK,LGA,PT11H34M,False,False,False,620.10,3310.0,1652880300||1652892600||1652908440,2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...,1652888160||1652902620||1652921940,2022-05-18T08:36:00.000-07:00||2022-05-18T13:3...,SEA||DEN||LGA,OAK||SEA||DEN,Alaska Airlines||Alaska Airlines||United,AS||AS||UA,Boeing 737-900||Boeing 737-800||Airbus A319,7860||10020||13500,672||1017||1621,coach||coach||coach
1,53c459f0331769569a77dd6c86461df4,2022-05-02,2022-06-26,OAK,EWR,PT16H5M,False,False,False,1301.10,3438.0,1656309300||1656338700||1656358020,2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...,1656325800||1656345480||1656367200,2022-06-27T06:30:00.000-04:00||2022-06-27T11:5...,DTW||ATL||EWR,OAK||DTW||ATL,Delta||Delta||United,DL||DL||UA,Boeing 737-900||Boeing 757||Boeing 737-800,16500||6780||9180,2072||604||762,coach||coach||coach
2,cdb75d4474097dbb19e2aa6d24be0482,2022-05-18,2022-07-15,OAK,DFW,PT13H56M,False,False,False,537.20,2335.0,1657897260||1657933500,2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...,1657905000||1657947420,2022-07-15T10:10:00.000-07:00||2022-07-15T23:5...,SEA||DFW,OAK||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737-900||Boeing 737-900,7740||13920,672||1663,coach||coach
3,d65efae6d5904c6474204ca0e6137547,2022-04-19,2022-05-22,OAK,DEN,PT8H26M,False,False,False,567.00,1689.0,1653254940||1653275700,2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...,1653262140||1653285300,2022-05-22T16:29:00.000-07:00||2022-05-22T23:5...,SEA||DEN,OAK||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Embraer 175||Boeing 737-800,7200||9600,672||1017,coach||coach
4,6199c40ac442ba05d387583954d33bc2,2022-05-02,2022-06-24,OAK,PHL,PT14H33M,False,False,False,866.71,2556.0,1656075600||1656100500||1656118740,2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...,1656082440||1656109500||1656127980,2022-06-24T08:54:00.000-06:00||2022-06-24T17:2...,SLC||MSP||PHL,OAK||SLC||MSP,Delta||Delta||Delta,DL||DL||DL,Airbus A319||Boeing 737-900||Airbus A321,6840||9000||9240,588||990||978,coach||coach||coach


In [4]:
df_cleaned = df.copy()

In [5]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   legId                              24000 non-null  object 
 1   searchDate                         24000 non-null  object 
 2   flightDate                         24000 non-null  object 
 3   startingAirport                    24000 non-null  object 
 4   destinationAirport                 24000 non-null  object 
 5   travelDuration                     24000 non-null  object 
 6   isBasicEconomy                     24000 non-null  bool   
 7   isRefundable                       24000 non-null  bool   
 8   isNonStop                          24000 non-null  bool   
 9   totalFare                          24000 non-null  float64
 10  totalTravelDistance                22286 non-null  float64
 11  segmentsDepartureTimeEpochSeconds  24000 non-null  obj

In [6]:
relevant_features = [
    'startingAirport',          # Categorical
    'destinationAirport',       # Categorical
    'travelDuration',           # Duration as time string, needs conversion
    'totalTravelDistance',      # Numerical
    'isNonStop',                # Binary
    'isBasicEconomy',           # Binary
    'segmentsDepartureTimeRaw',
    'segmentsCabinCode',        # Categorical for cabin class
    'segmentsAirlineName',      # Categorical
    'searchDate',               # Date, needs feature extraction
    'flightDate',               # Date, needs feature extraction
    'isRefundable',             # Binary
    'totalFare'                 # Target variable
]

In [7]:
df_cleaned = df_cleaned[relevant_features]

In [8]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   startingAirport           24000 non-null  object 
 1   destinationAirport        24000 non-null  object 
 2   travelDuration            24000 non-null  object 
 3   totalTravelDistance       22286 non-null  float64
 4   isNonStop                 24000 non-null  bool   
 5   isBasicEconomy            24000 non-null  bool   
 6   segmentsDepartureTimeRaw  24000 non-null  object 
 7   segmentsCabinCode         24000 non-null  object 
 8   segmentsAirlineName       24000 non-null  object 
 9   searchDate                24000 non-null  object 
 10  flightDate                24000 non-null  object 
 11  isRefundable              24000 non-null  bool   
 12  totalFare                 24000 non-null  float64
dtypes: bool(3), float64(2), object(8)
memory usage: 1.9+ MB


In [9]:
df_cleaned.head()

,startingAirport,destinationAirport,travelDuration,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,segmentsCabinCode,segmentsAirlineName,searchDate,flightDate,isRefundable,totalFare
0,OAK,LGA,PT11H34M,3310.0,False,False,2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...,coach||coach||coach,Alaska Airlines||Alaska Airlines||United,2022-05-01,2022-05-18,False,620.10
1,OAK,EWR,PT16H5M,3438.0,False,False,2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...,coach||coach||coach,Delta||Delta||United,2022-05-02,2022-06-26,False,1301.10
2,OAK,DFW,PT13H56M,2335.0,False,False,2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...,coach||coach,Alaska Airlines||Alaska Airlines,2022-05-18,2022-07-15,False,537.20
3,OAK,DEN,PT8H26M,1689.0,False,False,2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...,coach||coach,Alaska Airlines||Alaska Airlines,2022-04-19,2022-05-22,False,567.00
4,OAK,PHL,PT14H33M,2556.0,False,False,2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...,coach||coach||coach,Delta||Delta||Delta,2022-05-02,2022-06-24,False,866.71


In [10]:
# Function to convert duration from format 'PT#H#M' to total hours in float
def convert_duration_to_hours(duration):
    # Extract hours and minutes using regex
    hours = re.search(r'(\d+)H', duration)
    minutes = re.search(r'(\d+)M', duration)
    
    # Convert extracted values to integers, defaulting to 0 if not found
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    
    # Calculate total hours
    total_hours = hours + minutes / 60.0
    return total_hours

# Applying the conversion to the travelDuration column
df_cleaned['travelDurationHours'] = df_cleaned['travelDuration'].apply(convert_duration_to_hours)

# Output the first few rows to verify
df_cleaned[['travelDurationHours']].head()


,travelDurationHours
0,11.566667
1,16.083333
2,13.933333
3,8.433333
4,14.550000


In [11]:
df_cleaned = df_cleaned.drop(columns=['travelDuration'])
df_cleaned.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,segmentsCabinCode,segmentsAirlineName,searchDate,flightDate,isRefundable,totalFare,travelDurationHours
0,OAK,LGA,3310.0,False,False,2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...,coach||coach||coach,Alaska Airlines||Alaska Airlines||United,2022-05-01,2022-05-18,False,620.10,11.566667
1,OAK,EWR,3438.0,False,False,2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...,coach||coach||coach,Delta||Delta||United,2022-05-02,2022-06-26,False,1301.10,16.083333
2,OAK,DFW,2335.0,False,False,2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...,coach||coach,Alaska Airlines||Alaska Airlines,2022-05-18,2022-07-15,False,537.20,13.933333
3,OAK,DEN,1689.0,False,False,2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...,coach||coach,Alaska Airlines||Alaska Airlines,2022-04-19,2022-05-22,False,567.00,8.433333
4,OAK,PHL,2556.0,False,False,2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...,coach||coach||coach,Delta||Delta||Delta,2022-05-02,2022-06-24,False,866.71,14.550000


Extracting the number of flights (segments) taken to reach the destination from features like segmentsCabinCode or segmentsAirlineName could be quite helpful for improving the prediction accuracy of flight fares. Here’s why and how it can complement the existing isNonStop feature:

Why It’s Useful:
Granularity of Stops:

The isNonStop feature only indicates whether a flight has no stops (direct) or one or more stops (not direct). However, it does not specify the actual number of segments.
A flight with multiple layovers often has a higher operational cost and, in some cases, a lower ticket price due to added inconvenience, especially for economy tickets.
Impact on Fares:

Multi-segment flights often involve different pricing structures depending on the airlines and segments involved, potentially leading to a higher or lower overall fare.
Adding a feature indicating the number of segments can capture variations not reflected by isNonStop. For instance, a two-segment journey may be more or less costly than a three-segment one.
Model Interpretability:

By including a "number of segments" feature, the model gains better insight into the complexity and cost structure of each journey, potentially enhancing interpretability and accuracy.

In [12]:
# Function to count the number of segments in the 'segmentsCabinCode' column
def count_segments(cabin_code):
    # Count '||' to get the number of segments, adding 1 for the total count
    return cabin_code.count('||') + 1

# Applying the function to create the new 'numSegments' feature
df_cleaned['numSegments'] = df_cleaned['segmentsCabinCode'].apply(count_segments)

# Output first few rows to verify the new feature
df_cleaned[['segmentsCabinCode', 'numSegments']].head()

,segmentsCabinCode,numSegments
0,coach||coach||coach,3
1,coach||coach||coach,3
2,coach||coach,2
3,coach||coach,2
4,coach||coach||coach,3


In [13]:
df_cleaned[['isNonStop','segmentsCabinCode', 'numSegments']].tail(10)

,isNonStop,segmentsCabinCode,numSegments
23990,False,coach||coach,2
23991,True,coach,1
23992,False,coach||coach,2
23993,False,coach||coach,2
23994,False,coach||coach,2
23995,False,coach||coach,2
23996,False,coach||coach,2
23997,False,coach||coach,2
23998,False,coach||coach,2
23999,False,coach||coach,2


In [14]:
# Function to create a binary feature indicating if there are mixed cabin classes across segments
def has_mixed_cabin_classes(cabin_code):
    # Split by '||' and check if all elements are the same
    cabins = cabin_code.split('||')
    return 1 if len(set(cabins)) > 1 else 0

# Applying the function to create the new 'mixedCabinClasses' feature
df_cleaned['mixedCabinClasses'] = df_cleaned['segmentsCabinCode'].apply(has_mixed_cabin_classes)

# Function to create a binary feature indicating if multiple airlines are involved in the flight
def has_multiple_airlines(airline_name):
    # Split by '||' and check if all elements are the same
    airlines = airline_name.split('||')
    return 1 if len(set(airlines)) > 1 else 0

# Applying the function to create the new 'multipleAirlines' feature
df_cleaned['multipleAirlines'] = df_cleaned['segmentsAirlineName'].apply(has_multiple_airlines)

# Displaying the new features in the first 10 rows to verify
df_cleaned[['segmentsCabinCode', 'mixedCabinClasses', 'segmentsAirlineName', 'multipleAirlines']].head(10)


,segmentsCabinCode,mixedCabinClasses,segmentsAirlineName,multipleAirlines
0,coach||coach||coach,0,Alaska Airlines||Alaska Airlines||United,1
1,coach||coach||coach,0,Delta||Delta||United,1
2,coach||coach,0,Alaska Airlines||Alaska Airlines,0
3,coach||coach,0,Alaska Airlines||Alaska Airlines,0
4,coach||coach||coach,0,Delta||Delta||Delta,0
5,coach||coach||coach,0,Delta||United||United,1
6,coach||coach||coach,0,Delta||United||United,1
7,coach||coach||coach,0,Delta||United||United,1
8,coach||coach||coach,0,Delta||United||United,1
9,coach||coach||coach,0,Alaska Airlines||Alaska Airlines||United,1


In [15]:
df_cleaned[['segmentsCabinCode', 'mixedCabinClasses', 'segmentsAirlineName', 'multipleAirlines']].tail()

,segmentsCabinCode,mixedCabinClasses,segmentsAirlineName,multipleAirlines
23995,coach||coach,0,United||United,0
23996,coach||coach,0,American Airlines||American Airlines,0
23997,coach||coach,0,Delta||Delta,0
23998,coach||coach,0,Spirit Airlines||Spirit Airlines,0
23999,coach||coach,0,American Airlines||American Airlines,0


In [16]:
df_cleaned[['segmentsCabinCode','totalFare']].head(5)

,segmentsCabinCode,totalFare
0,coach||coach||coach,620.10
1,coach||coach||coach,1301.10
2,coach||coach,537.20
3,coach||coach,567.00
4,coach||coach||coach,866.71


In [17]:
df_cleaned['segmentsCabinCode'].value_counts()

segmentsCabinCode
coach||coach                           15163
coach                                   6683
coach||coach||coach                     1966
coach||coach||coach||coach                73
premium coach                             29
first||coach                              20
first||first                              15
premium coach||coach                      14
coach||premium coach                       8
first||first||first                        6
coach||first                               6
first                                      5
business                                   3
premium coach||premium coach               2
business||coach                            1
coach||coach||business                     1
first||coach||first                        1
premium coach||coach||coach                1
premium coach||premium coach||coach        1
first||coach||coach                        1
coach||first||first                        1
Name: count, dtype: int64

In [18]:
# Function to categorize cabin codes based on similarity or mixed types
def categorize_cabin_code(cabin_code):
    # Split the cabin_code by '||' to get individual cabin types
    cabins = cabin_code.split('||')
    # Check if all cabin types are the same, return that cabin type if true; otherwise, return 'mixed'
    return cabins[0] if len(set(cabins)) == 1 else 'mixed'

# Applying the function to create a categorized 'cabinType' feature
df_cleaned['cabinType'] = df_cleaned['segmentsCabinCode'].apply(categorize_cabin_code)

# Displaying the first few rows to verify the new 'cabinType' feature
df_cleaned[['segmentsCabinCode', 'cabinType']].head(10)


,segmentsCabinCode,cabinType
0,coach||coach||coach,coach
1,coach||coach||coach,coach
2,coach||coach,coach
3,coach||coach,coach
4,coach||coach||coach,coach
5,coach||coach||coach,coach
6,coach||coach||coach,coach
7,coach||coach||coach,coach
8,coach||coach||coach,coach
9,coach||coach||coach,coach


In [19]:
mixed_cabin_check = df_cleaned[df_cleaned['cabinType'] == 'mixed'][['segmentsCabinCode', 'cabinType']].head(10)
print(mixed_cabin_check)

         segmentsCabinCode cabinType
34         business||coach     mixed
4414          coach||first     mixed
4761          first||coach     mixed
5275          first||coach     mixed
5510  coach||premium coach     mixed
6397  premium coach||coach     mixed
6924          first||coach     mixed
7027  premium coach||coach     mixed
7852  premium coach||coach     mixed
7865          first||coach     mixed


In [20]:
df_cleaned.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,segmentsCabinCode,segmentsAirlineName,searchDate,flightDate,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType
0,OAK,LGA,3310.0,False,False,2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...,coach||coach||coach,Alaska Airlines||Alaska Airlines||United,2022-05-01,2022-05-18,False,620.10,11.566667,3,0,1,coach
1,OAK,EWR,3438.0,False,False,2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...,coach||coach||coach,Delta||Delta||United,2022-05-02,2022-06-26,False,1301.10,16.083333,3,0,1,coach
2,OAK,DFW,2335.0,False,False,2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...,coach||coach,Alaska Airlines||Alaska Airlines,2022-05-18,2022-07-15,False,537.20,13.933333,2,0,0,coach
3,OAK,DEN,1689.0,False,False,2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...,coach||coach,Alaska Airlines||Alaska Airlines,2022-04-19,2022-05-22,False,567.00,8.433333,2,0,0,coach
4,OAK,PHL,2556.0,False,False,2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...,coach||coach||coach,Delta||Delta||Delta,2022-05-02,2022-06-24,False,866.71,14.550000,3,0,0,coach


In [21]:
df_cleaned[['cabinType','totalFare']].value_counts()

cabinType  totalFare
coach      148.60       149
           408.60       143
           268.60       143
           351.60       142
           248.60       142
                       ... 
           223.08         1
           499.09         1
           222.70         1
           500.60         1
business   1317.60        1
Name: count, Length: 4808, dtype: int64

In [22]:
# Dropping the 'segmentsCabinCode' and 'segmentsAirlineName' features
df_cleaned = df_cleaned.drop(columns=['segmentsCabinCode', 'segmentsAirlineName'])

In [23]:
df_cleaned.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,searchDate,flightDate,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType
0,OAK,LGA,3310.0,False,False,2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...,2022-05-01,2022-05-18,False,620.10,11.566667,3,0,1,coach
1,OAK,EWR,3438.0,False,False,2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...,2022-05-02,2022-06-26,False,1301.10,16.083333,3,0,1,coach
2,OAK,DFW,2335.0,False,False,2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...,2022-05-18,2022-07-15,False,537.20,13.933333,2,0,0,coach
3,OAK,DEN,1689.0,False,False,2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...,2022-04-19,2022-05-22,False,567.00,8.433333,2,0,0,coach
4,OAK,PHL,2556.0,False,False,2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...,2022-05-02,2022-06-24,False,866.71,14.550000,3,0,0,coach


In [24]:
# Convert 'searchDate' and 'flightDate' to datetime format
df_cleaned['searchDate'] = pd.to_datetime(df_cleaned['searchDate'])
df_cleaned['flightDate'] = pd.to_datetime(df_cleaned['flightDate'])

# Calculate the days until departure
df_cleaned['daysUntilDeparture'] = (df_cleaned['flightDate'] - df_cleaned['searchDate']).dt.days

# Display the new feature along with search and flight dates to verify
print(df_cleaned[['searchDate', 'flightDate', 'daysUntilDeparture']].head(10))


  searchDate flightDate  daysUntilDeparture
0 2022-05-01 2022-05-18                  17
1 2022-05-02 2022-06-26                  55
2 2022-05-18 2022-07-15                  58
3 2022-04-19 2022-05-22                  33
4 2022-05-02 2022-06-24                  53
5 2022-04-29 2022-06-05                  37
6 2022-04-19 2022-04-26                   7
7 2022-04-19 2022-04-28                   9
8 2022-04-17 2022-06-07                  51
9 2022-04-18 2022-05-22                  34


In [25]:
df_cleaned = df_cleaned.drop(columns=['searchDate', 'flightDate'])

In [26]:
df_cleaned.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,segmentsDepartureTimeRaw,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType,daysUntilDeparture
0,OAK,LGA,3310.0,False,False,2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...,False,620.10,11.566667,3,0,1,coach,17
1,OAK,EWR,3438.0,False,False,2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...,False,1301.10,16.083333,3,0,1,coach,55
2,OAK,DFW,2335.0,False,False,2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...,False,537.20,13.933333,2,0,0,coach,58
3,OAK,DEN,1689.0,False,False,2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...,False,567.00,8.433333,2,0,0,coach,33
4,OAK,PHL,2556.0,False,False,2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...,False,866.71,14.550000,3,0,0,coach,53


In [27]:
# Check for missing data in the cleaned dataset
missing_data_summary = df_cleaned.isnull().sum()

# Display columns with missing values only
print(missing_data_summary[missing_data_summary > 0])

totalTravelDistance    1714
dtype: int64


In [28]:
# Removing rows with missing values in 'totalTravelDistance'
df_cleaned = df_cleaned.dropna(subset=['totalTravelDistance'])

# Confirming that there are no more missing values
print("Missing values in 'totalTravelDistance' after removal:", df_cleaned['totalTravelDistance'].isnull().sum())

Missing values in 'totalTravelDistance' after removal: 0


In [29]:
# Check for any remaining missing data in the dataset
remaining_missing_data = df_cleaned.isnull().sum()

# Display columns with missing values only
print(remaining_missing_data[remaining_missing_data > 0])

Series([], dtype: int64)


In [30]:
df_cleaned['segmentsDepartureTimeRaw'].head()

0    2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...
1    2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...
2    2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...
3    2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...
4    2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...
Name: segmentsDepartureTimeRaw, dtype: object

In [31]:
# Function to extract the hour and classify it into a time block
def get_time_block(departure_time_raw):
    # Extract the first time in the series (before '||')
    first_departure_time = departure_time_raw.split('||')[0]
    # Extract hour from the first time string using regex
    match = re.search(r'T(\d{2}):', first_departure_time)
    if match:
        hour = int(match.group(1))
        # Classify hour into time blocks
        if 5 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 21:
            return 'evening'
        else:
            return 'night'
    return 'unknown'  # For any format issues or missing data

# Apply the function to create a new 'timeBlock' column
df_cleaned['timeBlock'] = df_cleaned['segmentsDepartureTimeRaw'].apply(get_time_block)

# Displaying the result
df_cleaned[['segmentsDepartureTimeRaw', 'timeBlock']].head()


,segmentsDepartureTimeRaw,timeBlock
0,2022-05-18T06:25:00.000-07:00||2022-05-18T09:5...,morning
1,2022-06-26T22:55:00.000-07:00||2022-06-27T10:0...,night
2,2022-07-15T08:01:00.000-07:00||2022-07-15T18:0...,morning
3,2022-05-22T14:29:00.000-07:00||2022-05-22T20:1...,afternoon
4,2022-06-24T06:00:00.000-07:00||2022-06-24T13:5...,morning


In [32]:
df_cleaned['timeBlock'].value_counts()

timeBlock
morning      11041
afternoon     5908
evening       3575
night         1762
Name: count, dtype: int64

In [33]:
# Check for any remaining missing data in the dataset
remaining_missing_data = df_cleaned.isnull().sum()

# Display columns with missing values only
print(remaining_missing_data[remaining_missing_data > 0])

Series([], dtype: int64)


In [34]:
df_cleaned = df_cleaned.drop(columns=['segmentsDepartureTimeRaw'])

In [35]:
df_cleaned.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType,daysUntilDeparture,timeBlock
0,OAK,LGA,3310.0,False,False,False,620.10,11.566667,3,0,1,coach,17,morning
1,OAK,EWR,3438.0,False,False,False,1301.10,16.083333,3,0,1,coach,55,night
2,OAK,DFW,2335.0,False,False,False,537.20,13.933333,2,0,0,coach,58,morning
3,OAK,DEN,1689.0,False,False,False,567.00,8.433333,2,0,0,coach,33,afternoon
4,OAK,PHL,2556.0,False,False,False,866.71,14.550000,3,0,0,coach,53,morning


In [36]:
output_path = '/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT3/at_3/adv_mla_at3/data/df_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)

print("Data saved to:", output_path)

Data saved to: /Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT3/at_3/adv_mla_at3/data/df_cleaned.csv


In [37]:
df = df_cleaned.copy()

In [38]:
df.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType,daysUntilDeparture,timeBlock
0,OAK,LGA,3310.0,False,False,False,620.10,11.566667,3,0,1,coach,17,morning
1,OAK,EWR,3438.0,False,False,False,1301.10,16.083333,3,0,1,coach,55,night
2,OAK,DFW,2335.0,False,False,False,537.20,13.933333,2,0,0,coach,58,morning
3,OAK,DEN,1689.0,False,False,False,567.00,8.433333,2,0,0,coach,33,afternoon
4,OAK,PHL,2556.0,False,False,False,866.71,14.550000,3,0,0,coach,53,morning


In [39]:
from sklearn.preprocessing import LabelEncoder

# Combine unique values from both columns to ensure consistent encoding
all_airports = pd.concat([df['startingAirport'], df['destinationAirport']]).unique()

# Initialize and fit LabelEncoder with combined unique values
airport_encoder = LabelEncoder()
airport_encoder.fit(all_airports)

# Apply the encoder to both 'startingAirport' and 'destinationAirport'
df['startingAirport_encoded'] = airport_encoder.transform(df['startingAirport'])
df['destinationAirport_encoded'] = airport_encoder.transform(df['destinationAirport'])

# Verify encoding matches by printing first few rows of encoded columns
print(df[['startingAirport', 'startingAirport_encoded', 'destinationAirport', 'destinationAirport_encoded']].head())

  startingAirport  startingAirport_encoded destinationAirport  \
0             OAK                       12                LGA   
1             OAK                       12                EWR   
2             OAK                       12                DFW   
3             OAK                       12                DEN   
4             OAK                       12                PHL   

   destinationAirport_encoded  
0                          10  
1                           6  
2                           4  
3                           3  
4                          14  


In [40]:
df.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType,daysUntilDeparture,timeBlock,startingAirport_encoded,destinationAirport_encoded
0,OAK,LGA,3310.0,False,False,False,620.10,11.566667,3,0,1,coach,17,morning,12,10
1,OAK,EWR,3438.0,False,False,False,1301.10,16.083333,3,0,1,coach,55,night,12,6
2,OAK,DFW,2335.0,False,False,False,537.20,13.933333,2,0,0,coach,58,morning,12,4
3,OAK,DEN,1689.0,False,False,False,567.00,8.433333,2,0,0,coach,33,afternoon,12,3
4,OAK,PHL,2556.0,False,False,False,866.71,14.550000,3,0,0,coach,53,morning,12,14


In [41]:
# Applying label encoding for boolean features
df['isNonStop_encoded'] = df['isNonStop'].astype(int)
df['isBasicEconomy_encoded'] = df['isBasicEconomy'].astype(int)
df['isRefundable_encoded'] = df['isRefundable'].astype(int)

In [42]:
df.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType,daysUntilDeparture,timeBlock,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded
0,OAK,LGA,3310.0,False,False,False,620.10,11.566667,3,0,1,coach,17,morning,12,10,0,0,0
1,OAK,EWR,3438.0,False,False,False,1301.10,16.083333,3,0,1,coach,55,night,12,6,0,0,0
2,OAK,DFW,2335.0,False,False,False,537.20,13.933333,2,0,0,coach,58,morning,12,4,0,0,0
3,OAK,DEN,1689.0,False,False,False,567.00,8.433333,2,0,0,coach,33,afternoon,12,3,0,0,0
4,OAK,PHL,2556.0,False,False,False,866.71,14.550000,3,0,0,coach,53,morning,12,14,0,0,0


In [53]:
# Display unique values for 'startingAirport' and 'startingAirport_encoded'
unique_starting_airport = df['startingAirport'].unique()
unique_starting_airport_encoded = df['startingAirport_encoded'].unique()

print("Unique startingAirport values:", unique_starting_airport)
print("Unique startingAirport_encoded values:", unique_starting_airport_encoded)


Unique startingAirport values: ['OAK' 'EWR' 'JFK' 'MIA' 'BOS' 'CLT' 'PHL' 'ORD' 'DTW' 'IAD' 'ATL' 'SFO'
 'LAX' 'DEN' 'LGA' 'DFW']
Unique startingAirport_encoded values: [12  6  8 11  1  2 14 13  5  7  0 15  9  3 10  4]


In [54]:
# Display unique values for 'startingAirport' and 'startingAirport_encoded'
unique_des_airport = df['destinationAirport'].unique()
unique_des_airport_encoded = df['destinationAirport_encoded'].unique()

print("Unique startingAirport values:", unique_des_airport)
print("Unique startingAirport_encoded values:", unique_des_airport_encoded)

Unique startingAirport values: ['LGA' 'EWR' 'DFW' 'DEN' 'PHL' 'BOS' 'DTW' 'ATL' 'MIA' 'CLT' 'IAD' 'SFO'
 'JFK' 'LAX' 'ORD' 'OAK']
Unique startingAirport_encoded values: [10  6  4  3 14  1  5  0 11  2  7 15  8  9 13 12]


In [43]:
df['isNonStop_encoded'].value_counts()

isNonStop_encoded
0    15787
1     6499
Name: count, dtype: int64

In [44]:
# Initialize LabelEncoder for 'cabinType'
cabin_type_encoder = LabelEncoder()
df['cabinType_encoded'] = cabin_type_encoder.fit_transform(df['cabinType'])

# Verify the encoding by displaying the first few rows
df[['cabinType', 'cabinType_encoded']].head()

,cabinType,cabinType_encoded
0,coach,1
1,coach,1
2,coach,1
3,coach,1
4,coach,1


In [45]:
df['cabinType_encoded'].value_counts()

cabinType_encoded
1    22171
3       55
4       31
2       26
0        3
Name: count, dtype: int64

In [46]:
# Define a custom mapping for time blocks
time_block_mapping = {'morning': 0, 'afternoon': 1, 'evening': 2, 'night': 3}

# Apply the mapping to create the encoded 'timeBlock_encoded' column
df['timeBlock_encoded'] = df['timeBlock'].map(time_block_mapping)

# Display the encoded values to verify
print(df[['timeBlock', 'timeBlock_encoded']].head())

   timeBlock  timeBlock_encoded
0    morning                  0
1      night                  3
2    morning                  0
3  afternoon                  1
4    morning                  0


In [47]:
df.head()

,startingAirport,destinationAirport,totalTravelDistance,isNonStop,isBasicEconomy,isRefundable,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,cabinType,daysUntilDeparture,timeBlock,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded,cabinType_encoded,timeBlock_encoded
0,OAK,LGA,3310.0,False,False,False,620.10,11.566667,3,0,1,coach,17,morning,12,10,0,0,0,1,0
1,OAK,EWR,3438.0,False,False,False,1301.10,16.083333,3,0,1,coach,55,night,12,6,0,0,0,1,3
2,OAK,DFW,2335.0,False,False,False,537.20,13.933333,2,0,0,coach,58,morning,12,4,0,0,0,1,0
3,OAK,DEN,1689.0,False,False,False,567.00,8.433333,2,0,0,coach,33,afternoon,12,3,0,0,0,1,1
4,OAK,PHL,2556.0,False,False,False,866.71,14.550000,3,0,0,coach,53,morning,12,14,0,0,0,1,0


In [48]:
# Dropping categorical features
df_num = df.drop(columns=['startingAirport', 'destinationAirport','isNonStop','isBasicEconomy','isRefundable','cabinType','timeBlock'])

In [49]:
df_num.head()

,totalTravelDistance,totalFare,travelDurationHours,numSegments,mixedCabinClasses,multipleAirlines,daysUntilDeparture,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded,cabinType_encoded,timeBlock_encoded
0,3310.0,620.10,11.566667,3,0,1,17,12,10,0,0,0,1,0
1,3438.0,1301.10,16.083333,3,0,1,55,12,6,0,0,0,1,3
2,2335.0,537.20,13.933333,2,0,0,58,12,4,0,0,0,1,0
3,1689.0,567.00,8.433333,2,0,0,33,12,3,0,0,0,1,1
4,2556.0,866.71,14.550000,3,0,0,53,12,14,0,0,0,1,0


In [50]:
df_num = df_num.drop(columns=['mixedCabinClasses','multipleAirlines'])

In [51]:
df_num.head()

,totalTravelDistance,totalFare,travelDurationHours,numSegments,daysUntilDeparture,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded,cabinType_encoded,timeBlock_encoded
0,3310.0,620.10,11.566667,3,17,12,10,0,0,0,1,0
1,3438.0,1301.10,16.083333,3,55,12,6,0,0,0,1,3
2,2335.0,537.20,13.933333,2,58,12,4,0,0,0,1,0
3,1689.0,567.00,8.433333,2,33,12,3,0,0,0,1,1
4,2556.0,866.71,14.550000,3,53,12,14,0,0,0,1,0


In [52]:
output_path = '/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT3/at_3/adv_mla_at3/data/df_cleaned_num.csv'
df_num.to_csv(output_path, index=False)

print("Data saved to:", output_path)

Data saved to: /Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT3/at_3/adv_mla_at3/data/df_cleaned_num.csv
